In [103]:
import json
import pandas as pd 
from pymongo import MongoClient
import nltk
import numpy as np
from operator import itemgetter


In [88]:
with open("inverted_index.txt","r") as f:
    text=f.read()
with open("inverted_index2.txt","r") as f:
    text1=f.read()

In [89]:
inverted_index=json.loads(text)
inverted_index2=json.loads(text1)

In [138]:
def stemming_query(inputs):
    words=nltk.word_tokenize(inputs)
    words=[word.lower() for word in words if word.isalnum()]
    lemantizer=nltk.stem.WordNetLemmatizer()
    words =[lemantizer.lemmatize(word) for word in words]
    
    return words

def query_score(queries):
    score_dic=[]
    intersection=set(inverted_index[queries[0]])
    counter=0
    #calculates tf-idf for the que
    try:
        for query in queries:
            if counter == 0:
                idf=1+np.log(29169/len(inverted_index[query]))
                score_dic.append(idf * (queries.count(query)/len(queries)))
                counter +=1 
            else:
                idf=1+np.log(29169/len(inverted_index[query]))
                score_dic.append(idf * (queries.count(query)/len(queries)))
                intersection=intersection.intersection(inverted_index[query])
        score_dic=np.array(score_dic)
    except ZeroDivisionError as e1:
        print("Query not found in the inverted_index")
        sys.exit(1)
    return (list(intersection),score_dic)
def manhattan_distance(documents,query_score):
    ranking_list=[]
    for document in documents:
        doc_score=[]
        for query in queries:
            doc_score.append(inverted_index2[query][document])
        doc_score=np.array(doc_score)
        ranking_list.append((document,np.sum(np.abs(query_score-doc_score))))
    
    ranking_list=sorted(ranking_list, key=itemgetter(1),reverse=True)
    return ranking_list
def show_related_data(ranked_list):
    last_data=pd.DataFrame(columns=["Title","Url","Manhattan similiraties"])
    for i in ranked_list:
        doc_num=(int(i[0][9:]))
        dataframe=pd.read_csv("./tsvfiles/tsv_file{}.txt".format(doc_num),sep="\t")
        dataframe["Manhattan similiraties"]=[i[1] for i in ranked_list]
        last_data.append(dataframe[["film_name","URL","Manhattan similiraties"]])
    return last_data

In [140]:
queries=stemming_query(input())
score=query_score(queries)
score=manhattan_distance(score[0],score[1])
score

disney 2019


[('document_11903', 6.114159279756877),
 ('document_11910', 6.034805506057101),
 ('document_10852', 6.017143737610619)]

In [21]:
client= MongoClient()
db=client["HW3"]
collection = db["Box-infoandRest"]
data=collection.find()

In [22]:
dataframe=pd.DataFrame(data)
dataframe.drop(["_id"],axis=1,inplace=True)

In [33]:
dataframe.columns.values

array(['id', 'film_name', 'Directed by', 'Produced by', 'Written by',
       'Starring', 'Music by', 'Based on', 'Cinematography',
       'Productioncompany ', 'Release date', 'Running time', 'Country',
       'Language', 'Budget', 'URL', 'Title', 'Intro', 'Plot',
       'Title_tokenized', 'Intro_tokenized', 'Plot_tokenized',
       'Title_lemmatize', 'Intro_lemmatize', 'Plot_lemmatize', 'Title_sw',
       'Intro_sw', 'Plot_sw', 'Title_punc', 'Intro_punc', 'Plot_punc'],
      dtype=object)

In [85]:
columns=['film_name', 'Directed by', 'Produced by', 'Written by',
       'Starring', 'Music by', 'Based on', 'Cinematography',
       'Productioncompany ', 'Release date', 'Running time', 'Country',
       'Language', 'Budget', 'URL']
dataframe["new"]=np.nan
for col in columns:
    dataframe["new"] = dataframe["new"].astype(str) + " "+ dataframe[col].astype(str)

In [87]:
dataframe["new"].apply(stemming_query)


0        [nan, love, by, the, light, of, the, moon, edw...
1        [nan, the, martyred, president, edwin, porter,...
2        [nan, jack, and, the, beanstalk, george, flemi...
3        [nan, alice, in, wonderland, cecil, hepworthpe...
4        [nan, the, great, train, robbery, edwin, porte...
                               ...                        
29164    [nan, paarai, none, sarath, kumarjayarammeenar...
29165    [nan, jayam, raja, varalakshmieditor, mohan, p...
29166    [nan, kaadhal, kondein, selvaraghavan, vimalag...
29167    [nan, kovilpatti, veeralakshmi, rajeshwar, non...
29168    [nan, whistle, jerry, medium, dream, sujatha, ...
Name: new, Length: 29169, dtype: object

2

In [ ]:
last_data=pd.DataFrame(columns=["Title","Url","Manhattan similiraties"])

In [135]:
pd.DataFrame([[1,2,3]],columns=last_data.columns)

,Title,Url,Manhattan similiraties
0,1,2,3
